Stella MIRRI,  Jack EVANS
# Mini projet 1 :   Rangement d’un tuyau de jardin

## 1) Etude du probleme d'optimisation

1. Interpréter le cout $(4)$. Que cherche-t-on a minimiser?

***
**REPONSE**

On cherche a minimiser : 
- d'une part   $(x_N-\frac{L}{2})^2$   pour faire avoir   $x_N=\frac{L}{2}$   ou plutôt avoir une valeur *suffisamment proche* pour faire l'approximation en pratique (d'où le $K$ grand pour donner plus de poids a ce terme).
- d'autre part   $ds\sum{|y_i|}=\frac{L}{N}\sum{|y_i|}$   pour que le tuyau soit le "plus proche possible" de l'axe des abscisses $y=0$ (faire 2 dessins)


***

2. Pourquoi n’a-t-on pas plutôt imposé la contrainte $x_N = \frac{L}{2}$ ? (Indice : on pourra considérer la valeur initiale qu’il faudrait alors fournir au solveur pour la résolution numérique d’un tel problème.)

***
**REPONSE**


***

3. Quelle difficulté peut présenter le coût $(4)$ numériquement ?

***
**REPONSE**


***

4. Justifier que minimiser $(4)$ est équivalent à minimiser
$$K(x_N − \frac{L}{2})^2+ ds\sum{wi}$$
t.q. $wi ≥ yi$ et $wi ≥ −yi , i = 0, . . . , N (5)$
où $wi (i = 0, . . . , N )$ sont de nouvelles variables scalaires. Quel est l’avantage de cette réécriture ? On la retiendra pour la suite du sujet.

***
**REPONSE**


***

5. Formuler le problème d’optimisation à résoudre sous la forme
$$min_{(c_{eq}(z)=0 , \ c_{ineq}(z)≤0)} f(z) \qquad (6)$$
On précisera les variables de décision $z$, leur nombre $n$, les contraintes $ceq$ et $cineq$ ainsi que la fonction objectif $f$ à minimiser

***
**REPONSE**


***